In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Edit inputs as comma seperated values, leave as empty strings to include all data.
#Inputs are tournament_ids or team_ids
tournament_input = '' #'103540363364808496, 104174613295388764, 104841804583318464, 105522984810490982, 106269654659501670, 106926282333089592, 107418445247362001, 108195478954601542, 108998961191900167, 109625152894842837, 110198981276611770, 110371551277508787'

team_input = '' #'100205573495116443,  99566404852189289,  99566404853854212, 99566404579461230,  98767991853197861,  99566404850008779, 99566404853058754,  98767991892579754,  98767991926151025, 99566404854685458,  98767991882270868, 100725845018863243, 100205573496804586,  98767991877340524,  99566404848691211' 


In [3]:
#Importing Data 
tourn_csv_data = pd.read_csv('data\\tournaments.csv')[['id','leagueid', 'slug']]
tourn_json_data = pd.read_json('data\\tournaments.jsonl', lines = True).drop(['id', 'leagueId'], axis = 1)
tourn_data = tourn_csv_data.merge(tourn_json_data, on = 'slug')
teams_data = pd.read_csv('data\\teams.csv').set_index('team_id')
match_data = pd.read_csv("data\\match_history.csv").rename({'gameid': 'platformgameid'}, axis = 1)
stats_data = pd.read_csv('data\\gold_dif.csv').sort_values('platformgameid')
league_data = pd.read_csv("data\\leagues.csv").rename({'name': 'Region', 'id': 'leagueid'}, axis = 1)

# gold_df = pd.read_csv('data\\gold_differences.csv').set_index('platformgameid')

region_tiers = {'t1': {'regions': ['LPL', 'LCK', 'Worlds', 'MSI'],
                        'start_elo': 1500},
                't2': {'regions': ['LCS', 'LEC'],
                        'start_elo': 1250},
                't3': {'regions' : [],
                        'start_elo': 1000}}

In [4]:
display(tourn_data.head())

,id,leagueid,slug,name,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,nacl_qualifiers_2_summer_2023,#2 Summer 2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,gll_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,elite_series_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,superliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,ultraliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [5]:
#Filter leagues to user input
tourn_data.head()

,id,leagueid,slug,name,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,nacl_qualifiers_2_summer_2023,#2 Summer 2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,gll_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,elite_series_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,superliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,ultraliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [6]:
#Fixing game data, game time/duration
match_data['event_time'] = pd.to_datetime(match_data['event_time'])
match_data['game_time'] /= 60
match_data['game_time'] = match_data['game_time'].apply(lambda x: f"{str(x)[:2]}:{int((float(str(x)[2:])*(6/100))):02}")
match_data['event_time'] = pd.to_datetime(match_data['event_time'])
# match_data = match_data.loc[match_data['event_time'] > ('2023')]
match_data['event_time'] = match_data['event_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))

tourn_df = league_data[['leagueid', 'Region']].merge(tourn_data, how='inner', on = 'leagueid')

display(tourn_data.head())
display(match_data.head())
display(tourn_df.head())
display(league_data.head())

,id,leagueid,slug,name,sport,startDate,endDate,stages
0,110733838935136200,109518549825754242,nacl_qualifiers_2_summer_2023,#2 Summer 2023,lol,2023-07-20,2023-08-15,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
1,110428723804419399,105266108767593290,gll_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,110424377524465827,107407335299756365,elite_series_summer_2023,Summer 2023,lol,2023-06-04,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
3,110349992504762921,105266074488398661,superliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-19,"[{'name': 'Regular Season', 'type': None, 'slu..."
4,110310652404523974,105266088231437431,ultraliga_summer_2023,Summer 2023,lol,2023-05-28,2023-08-13,"[{'name': 'Regular Season', 'type': None, 'slu..."


,platformgameid,egameid,event_time,game_time,blue_team_id,red_team_id,winning_side,winning_team
0,ESPORTSTMNT01:1120735,101978171977781193,2019-05-06 12:27,24:48,98767991902391669,101978171843206569,100,98767991902391669
1,ESPORTSTMNT01:1132954,102181142520557227,2019-06-12 13:19,23:19,99871276356460471,100725845022060229,200,100725845022060229
2,ESPORTSTMNT01:1121419,101978171965329317,2019-05-13 15:25,26:32,100783238110621058,98767991853197861,200,98767991853197861
3,ESPORTSTMNT01:1161821,102147202193408304,2019-07-26 01:10,26:26,99566405123587075,99566405132177721,100,99566405123587075
4,ESPORTSTMNT01:1110177,101978171979157453,2019-05-01 11:45,26:13,100783238110621058,99566408358330419,100,100783238110621058


,leagueid,Region,id,slug,name,sport,startDate,endDate,stages
0,98767991299243165,LCS,105658534671026792,lcs_summer_2021,LCS Summer 2021,lol,2021-06-04,2021-08-30,"[{'name': 'Regular Season', 'type': None, 'slu..."
1,98767991299243165,LCS,108206581962155974,lcs_summer_2022,LCS Summer 2022,lol,2022-06-16,2022-09-13,"[{'name': 'Regular Season', 'type': None, 'slu..."
2,98767991299243165,LCS,107458335260330212,lcs_lock_in_2022,LCS Lock In 2022,lol,2022-01-14,2022-01-31,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
3,98767991299243165,LCS,105522217230238828,lcs_lock_in_2021,LCS Lock In 2021,lol,2021-01-15,2021-01-31,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
4,98767991299243165,LCS,110303581083678395,lcs_summer_2023,Summer 2023,lol,2023-05-31,2023-08-22,"[{'name': 'Regular Season', 'type': None, 'slu..."


,leagueid,Region,slug,sport,image,lightimage,darkimage,region,priority,displaypriority,tournaments
0,98767991299243165,LCS,lcs,lol,http://static.lolesports.com/leagues/LCSNew-01...,http://static.lolesports.com/leagues/LCSNew-01...,http://static.lolesports.com/leagues/LCSNew-01...,NORTH AMERICA,1,"{position=0, status=selected}","[{id=110303581083678395}, {id=1095170900666056..."
1,109511549831443335,LCS Challengers,north_american_challenger_league,lol,http://static.lolesports.com/leagues/167112649...,http://static.lolesports.com/leagues/167112649...,http://static.lolesports.com/leagues/167112649...,NORTH AMERICA,1000,"{position=1, status=selected}","[{id=110574243270525539}, {id=1095186976607462..."
2,109518549825754242,LCS Challengers Qualifiers,lcs_challengers_qualifiers,lol,http://static.lolesports.com/leagues/167112651...,http://static.lolesports.com/leagues/167112651...,http://static.lolesports.com/leagues/167112651...,NORTH AMERICA,1000,"{position=2, status=selected}","[{id=110894544221659776}, {id=1107338389351362..."
3,107898214974993351,College Championship,college_championship,lol,http://static.lolesports.com/leagues/164639609...,http://static.lolesports.com/leagues/164639609...,http://static.lolesports.com/leagues/166513890...,NORTH AMERICA,1000,"{position=3, status=selected}",[{id=107898708099217418}]
4,98767991332355509,CBLOL,cblol-brazil,lol,http://static.lolesports.com/leagues/cblol-log...,http://static.lolesports.com/leagues/cblol-log...,http://static.lolesports.com/leagues/cblol-log...,BRAZIL,204,"{position=0, status=not_selected}","[{id=110413046183015975}, {id=1095236414167506..."


In [7]:
def parse_string_to_dict_list(string):
    dict_list = []
    dict_strs = string.split("}, {")
    for dict_str in dict_strs:
        dict_items = dict_str.replace('[', '').replace(']', '').replace('{', '').replace('}', '').split(", ")
        new_dict = {}
        for dict_item in dict_items:
            key, value = dict_item.split("=")
            new_dict[key] = int(value)
        dict_list.append(new_dict)
    return dict_list

stats_data['teams'] = stats_data['teams'].apply(lambda x: parse_string_to_dict_list(x))

lane_df = pd.DataFrame(
    columns=[ "platformgameid",
        "blue_team_gold", "red_team_gold",
        "blue_team_gold_lead", "red_team_gold_lead",
        "blue_team_towers", "red_team_towers",
        "blue_team_dragons", "red_team_dragons",
        "blue_team_barons", "red_team_barons",
        "blue_team_kills", "red_team_kills",
        "blue_team_deaths", "red_team_deaths",
        "blue_team_assists", "red_team_assists",
    ]
)

for match in stats_data.loc[stats_data["game_event"] == "lane_state"].itertuples():
    lane_df = lane_df.append(
        {
            "platformgameid" : match.platformgameid,
            "blue_team_gold": match.teams[0]["totalgold"],
            "blue_team_gold_lead": match.teams[0]["totalgold"] - match.teams[1]["totalgold"],
            "blue_team_towers": match.teams[0]["towerkills"],
            "blue_team_dragons": match.teams[0]["dragonkills"],
            "blue_team_barons": match.teams[0]["baronkills"],
            "blue_team_kills": match.teams[0]["championskills"],
            "blue_team_deaths": match.teams[0]["deaths"],
            "blue_team_assists": match.teams[0]["assists"],

            "red_team_gold": match.teams[1]["totalgold"],
            "red_team_gold_lead": match.teams[1]["totalgold"] - match.teams[0]["totalgold"],
            "red_team_towers": match.teams[1]["towerkills"],
            "red_team_dragons": match.teams[1]["dragonkills"],
            "red_team_barons": match.teams[1]["baronkills"],
            "red_team_kills": match.teams[1]["championskills"],
            "red_team_deaths": match.teams[1]["deaths"],
            "red_team_assists": match.teams[1]["assists"],
        },
        ignore_index=True,
    ).reset_index(drop=True)
lane_df.head()

,platformgameid,blue_team_gold,red_team_gold,blue_team_gold_lead,red_team_gold_lead,blue_team_towers,red_team_towers,blue_team_dragons,red_team_dragons,blue_team_barons,red_team_barons,blue_team_kills,red_team_kills,blue_team_deaths,red_team_deaths,blue_team_assists,red_team_assists
0,ESPORTSTMNT01:1110148,25282,21708,3574,-3574,0,0,2,0,0,0,10,5,5,10,15,6
1,ESPORTSTMNT01:1110162,24729,22639,2090,-2090,1,0,0,1,0,0,3,5,5,3,2,8
2,ESPORTSTMNT01:1110177,26627,21812,4815,-4815,1,0,1,0,0,0,12,4,4,12,21,4
3,ESPORTSTMNT01:1110187,26205,24211,1994,-1994,0,0,0,1,0,0,7,5,5,7,17,8
4,ESPORTSTMNT01:1110198,22482,23788,-1306,1306,0,0,0,1,0,0,5,5,5,5,9,12


In [8]:

match_data = match_data.merge(lane_df[['platformgameid', 'blue_team_gold', 'blue_team_gold_lead', 'blue_team_dragons', 'blue_team_kills', 'blue_team_deaths'
                          , 'red_team_gold', 'red_team_gold_lead', 'red_team_dragons', 'red_team_kills', 'red_team_deaths']], how = 'inner', on = 'platformgameid').set_index('egameid')

In [9]:
display(match_data.head())

,platformgameid,event_time,game_time,blue_team_id,red_team_id,winning_side,winning_team,blue_team_gold,blue_team_gold_lead,blue_team_dragons,blue_team_kills,blue_team_deaths,red_team_gold,red_team_gold_lead,red_team_dragons,red_team_kills,red_team_deaths
egameid,,,,,,,,,,,,,,,,,
101978171977781193,ESPORTSTMNT01:1120735,2019-05-06 12:27,24:48,98767991902391669,101978171843206569,100,98767991902391669,25554,4678,1,6,1,20876,-4678,0,1,6
102181142520557227,ESPORTSTMNT01:1132954,2019-06-12 13:19,23:19,99871276356460471,100725845022060229,200,100725845022060229,21416,-2877,0,2,2,24293,2877,1,2,2
101978171965329317,ESPORTSTMNT01:1121419,2019-05-13 15:25,26:32,100783238110621058,98767991853197861,200,98767991853197861,19963,-5670,0,1,9,25633,5670,1,9,1
102147202193408304,ESPORTSTMNT01:1161821,2019-07-26 01:10,26:26,99566405123587075,99566405132177721,100,99566405123587075,26335,6451,0,10,2,19884,-6451,1,2,10
101978171979157453,ESPORTSTMNT01:1110177,2019-05-01 11:45,26:13,100783238110621058,99566408358330419,100,100783238110621058,26627,4815,1,12,4,21812,-4815,0,4,12


In [10]:
mh_df = pd.DataFrame(columns = ['game_id', 'tournament_id', 'league_id', 'blue_team_id', 'red_team_id', 'winning_side'])

for tournament in tourn_df.itertuples():
    # print([len(stage['sections']) for stage in tournament.stages])
    for stage in tournament.stages:
        #Standardize Stage names
        stage['name'] = ' '.join([word.capitalize() for word in stage['name'].replace(' ', '_').split('_')])
        # print(stage['name'])
        for match in stage['sections'][0]['matches']:
            for game in match['games']:
                if(game['state'] == 'completed'):
                    #Remove region information from the tournament name in order to standardize tournament names
                    split = ' '.join([word if word.lower() not in league_data['Region'].apply(lambda x: x.lower()).values else '' for word in ''.join([i for i in tournament.name if not i.isdigit()]).replace('_', ' ').split(' ')])
                    year = ''.join([i for i in tournament.name if i.isdigit()])

                    mh_df = mh_df.append({'game_id': int(game['id']), 
                                          'game_num': int(game['number']),
                                        'tournament_id': int(tournament.id),
                                        'tournament': f"{tournament.Region} {split}{stage['name']} {year}",
                                        'stage': stage['name'],
                                        'region': tournament.Region,
                                        'league_id': int(tournament.leagueid),
                                        'blue_team_id': int(game['teams'][0]['id']), 
                                        'red_team_id': int(game['teams'][1]['id']),
                                        'winning_side': 100 if game['teams'][0]['result']['outcome'] == 'win' else 200}, ignore_index=True)
                                        # 'winning_team_id': int(game['teams'][0]['id']) if game['teams'][0]['result']['outcome'] == 'win' else int(game['teams'][1]['id'])}, ignore_index=True)


In [11]:
team_rep = 'acronym'

mh_df = mh_df.merge(teams_data[f"{team_rep}"], left_on = 'blue_team_id', right_index = True, how = 'inner').rename({f"{team_rep}":'blue_side'}, axis = 1)
mh_df = mh_df.merge(teams_data[f"{team_rep}"], left_on = 'red_team_id', right_index = True, how = 'inner').rename({f"{team_rep}":'red_side'}, axis = 1)
# mh_df = mh_df.merge(teams_df[f"{team_rep}"], left_on = 'winning_team_id', right_index = True, how = 'inner').rename({f"{team_rep}":'winner'}, axis = 1)
mh_df = mh_df.merge(match_data[['platformgameid', 'event_time', 'game_time'
                                , 'blue_team_gold', 'blue_team_gold_lead', 'blue_team_dragons', 'blue_team_kills', 'blue_team_deaths'
                                , 'red_team_gold', 'red_team_gold_lead', 'red_team_dragons', 'red_team_kills', 'red_team_deaths']], left_on = 'game_id', right_index = True, how = 'inner')
mh_df[['winning_side', 'game_num']] = mh_df[['winning_side', 'game_num']].astype(int)

mh_df = mh_df.drop_duplicates('platformgameid').sort_values(by = 'event_time', ascending = True)
teams = set(sorted(set(mh_df['blue_team_id'].values)) +  sorted(set(mh_df['red_team_id'].values)))

In [12]:
team_stats_df = pd.DataFrame(columns = ['teamid', 'matches', 'avg_gold_lead_14', 'avg_gold_14', 'avg_team_deaths_14', 'avg_team_dragons_14', 'avg_team_kills_14'])

for team in teams:
    team_match_df_blue = mh_df.loc[mh_df['blue_team_id'].astype(str) == f'{team}']
    team_match_df_red = mh_df.loc[mh_df['red_team_id'].astype(str) == f'{team}']
    matches = len(team_match_df_blue.index) + len(team_match_df_red.index)
    if matches > 0:
        team_stats_df = team_stats_df.append({'teamid' : str(team), 
                          'matches': matches,
                          'avg_gold_lead_14': (team_match_df_blue['blue_team_gold_lead'].astype(float).sum() + team_match_df_red['red_team_gold_lead'].astype(float).sum()) / matches ,
                          'avg_gold_14': (team_match_df_blue['blue_team_gold'].sum() + team_match_df_red['red_team_gold'].sum()) / matches,
                          'avg_team_deaths_14': (team_match_df_blue['blue_team_deaths'].sum() + team_match_df_red['red_team_deaths'].sum()) / matches,
                          'avg_team_dragons_14': (team_match_df_blue['blue_team_dragons'].sum() + team_match_df_red['red_team_dragons'].sum()) / matches,
                          'avg_team_kills_14': (team_match_df_blue['blue_team_kills'].sum() + team_match_df_red['red_team_kills'].sum()) / matches,
                          }, ignore_index = True)
team_stats_df = team_stats_df.set_index('teamid')

In [13]:
team_stats_df.sort_values('avg_gold_14').head()

,matches,avg_gold_lead_14,avg_gold_14,avg_team_deaths_14,avg_team_dragons_14,avg_team_kills_14
teamid,,,,,,
105193571520283762,1,-9383.000000,20106.000000,26.000000,0.000000,5.000000
105351737774956843,1,-5320.000000,20969.000000,9.000000,1.000000,3.000000
109697686032800938,14,-3393.857143,21321.214286,7.071429,0.428571,3.857143
105549984475833252,10,-1750.600000,21422.000000,3.500000,0.500000,1.800000
103877756742242918,3,241.333333,21459.333333,2.666667,0.666667,3.000000


In [14]:

#Home regions are defined as the region in which a team plays the most games, discounting international events
def find_home_region(team):
    region_counts = mh_df.loc[(mh_df['blue_team_id'] == team.name) | (mh_df['red_team_id'] == team.name), 'region'].value_counts().index
    if len(region_counts) > 1:
        if region_counts[0] not in ('MSI', 'Worlds'):
            return region_counts[0]
        elif region_counts[1] not in ('MSI', 'Worlds'):
            try:
                return region_counts[1]
            except:
                return 'Wildcard'
        else:
            try:
                return region_counts[2]
            except:
                return 'Wildcard'
    else:
        return region_counts[0] if region_counts[0] not in ('MSI', 'Worlds') else 'Wildcard'
    
elo_df = teams_data.loc[teams]
elo_df['region'] = elo_df[[f'{team_rep}']].apply(lambda x: find_home_region(x), axis = 1)

#Regional stat distributions are used to determine how a given team performs against their competition
region_stats_df = pd.DataFrame(columns = ['region', 'avg_gold_lead_14', 'std_gold_lead_14', 'avg_gold_14', 'std_gold_14',
                                           'avg_team_deaths_14', 'std_team_deaths_14', 'avg_team_dragons_14', 'std_team_dragons_14', 'avg_team_kills_14', 'std_team_kills_14'])

#Populate regional stat distributions
for region in elo_df['region'].drop_duplicates().values:
    region_teams = team_stats_df.loc[[str(index) for index in elo_df.loc[elo_df['region'] == region].index]]
    region_stats_df = region_stats_df.append({'region' : region, 
                                                'avg_gold_lead_14': region_teams['avg_gold_lead_14'].mean(), 
                                                'std_gold_lead_14': region_teams['avg_gold_lead_14'].std(),
                                                'avg_gold_14': region_teams['avg_gold_14'].mean() , 
                                                'std_gold_14': region_teams['avg_gold_14'].std(),
                                                'avg_team_deaths_14': region_teams['avg_team_deaths_14'].mean(), 
                                                'std_team_deaths_14': region_teams['avg_team_deaths_14'].std(), 
                                                'avg_team_dragons_14': region_teams['avg_team_dragons_14'].mean(), 
                                                'std_team_dragons_14': region_teams['avg_team_dragons_14'].std(), 
                                                'avg_team_kills_14': region_teams['avg_team_kills_14'].mean(), 
                                                'std_team_kills_14': region_teams['avg_team_kills_14'].std()}, ignore_index = True)
region_stats_df = region_stats_df.set_index('region', drop = True)

#Fetch starting elo as defined as an initial starting elo based on Regional Strength of Competition, then adjusted per team based on their laning performance against the region as a whole.
def get_start_elo(team, region):
    team = str(team)
    for tier, data in region_tiers.items():
        if region in data['regions']:
            start_elo = data['start_elo']
            std_elo_cost = 20
            #Elo weighting costs from gold difference machine learning in gold_difference.ipynb. Capped each value to avoid edge cases where a team significantly outperformed in a given metric.
            elo_cost = {'gold_lead_14': min([0.57 * (team_stats_df.at[team, 'avg_gold_lead_14'] / region_stats_df.at[region, 'std_gold_lead_14']) * std_elo_cost, 50]),
                        'gold_14': min([0.24 * (team_stats_df.at[team, 'avg_gold_14'] / region_stats_df.at[region, 'std_gold_14']) * std_elo_cost, 50]), 
                        'team_deaths_14': -min([0.08 * (team_stats_df.at[team, 'avg_team_deaths_14'] / region_stats_df.at[region, 'std_team_deaths_14']) * std_elo_cost, 30]), 
                        'team_dragons_14': min([0.06 * (team_stats_df.at[team, 'avg_team_dragons_14'] / region_stats_df.at[region, 'std_team_dragons_14']) * std_elo_cost, 30]), 
                        'team_kills_14': min([0.06 * (team_stats_df.at[team, 'avg_team_kills_14'] / region_stats_df.at[region, 'std_team_kills_14']) * std_elo_cost, 30]), }
            start_elo += min(sum(elo_cost.values()), 500)
            return start_elo
    return 1000

            
elo_df['elo'] = elo_df[['region']].apply(lambda x: get_start_elo(x.name, x['region']), axis = 1)
elo_df = elo_df.drop_duplicates().sort_index()

In [15]:
elo_df.sort_values('elo', ascending=False).head(20)

,name,acronym,slug,region,elo
team_id,,,,,
98767991892579754,Royal Never Give Up,RNG,royal-never-give-up,LPL,1582.956507
100725845018863243,Dplus Kia,DK,dwg-kia,LCK,1578.131425
99566404853854212,Bilibili Gaming Pingan Bank,BLG,bilibili-gaming,LPL,1576.444418
98767991853197861,T1,T1,t1,LCK,1576.417027
100205573495116443,Gen.G,GEN,geng,LCK,1576.039875
99566404850008779,Suzhou LNG Esports,LNG,lng-esports,LPL,1574.002342
99566404853058754,WeiboGaming FAW AUDI,WBG,weibo-gaming,LPL,1573.267197
99566404579461230,kt Rolster,KT,kt-rolster,LCK,1565.410504
99566404852189289,Beijing JDG Intel Esports Club,JDG,jd-gaming,LPL,1565.180930


In [16]:
#Show all tournaments a given team participated in(in this case GenG)
print(mh_df['tournament_id'].loc[(mh_df['blue_team_id'] == 100205573495116443) | (mh_df['red_team_id'] == 100205573495116443)].drop_duplicates().to_list())

display(mh_df[['platformgameid', 'tournament_id', 'game_id', 'region', 'tournament', 'stage', 'event_time', 'game_time', 'blue_side', 'red_side', 'winning_side']].loc[mh_df['blue_side'].str.contains('G2') | mh_df['red_side'].str.contains('G2')].head())

print(teams)

[103540363364808496, 104174613295388764, 104841804583318464, 105522984810490982, 106269654659501670, 106926282333089592, 107418445247362001, 108195478954601542, 108998961191900167, 109625152894842837, 110198981276611770, 110371551277508787]


,platformgameid,tournament_id,game_id,region,tournament,stage,event_time,game_time,blue_side,red_side,winning_side
5671,ESPORTSTMNT05:1390161,103462459318635408,103462460596524867,LEC,LEC Spring Regular Season 2020,Regular Season,2020-01-24 17:39,34:19,G2,MAD,100
5645,ESPORTSTMNT05:1390263,103462459318635408,103462460596524815,LEC,LEC Spring Regular Season 2020,Regular Season,2020-01-25 20:18,30:53,SK,G2,200
5681,ESPORTSTMNT04:1120150,103462459318635408,103462460596590423,LEC,LEC Spring Regular Season 2020,Regular Season,2020-01-31 19:31,23:01,VIT,G2,200
5653,ESPORTSTMNT04:1130221,103462459318635408,103462460596524831,LEC,LEC Spring Regular Season 2020,Regular Season,2020-02-01 20:42,35:37,G2,AST,100
5666,ESPORTSTMNT04:1120649,103462459318635408,103462460596524857,LEC,LEC Spring Regular Season 2020,Regular Season,2020-02-07 21:29,38:45,KOI,G2,200


{104843437139220481, 99566404853854212, 99566404848691211, 107599485973747723, 108573042084687884, 109675484017801229, 108160026618060825, 107582178455767077, 105503778424809514, 99566408358330419, 105357804699574324, 99566405124618295, 105520713962793016, 99124844353216570, 105593161016793151, 103461966951059521, 108361563004344389, 105503848339138647, 103461966965149786, 110534732558665825, 103877976717529187, 99566405126602853, 99566404852189289, 103461966971048042, 110733877357854829, 99566404579461230, 103461966975897718, 110733889090164854, 108404211655217274, 105519846782566525, 105520784254806142, 105593182576105603, 103461966981927044, 98767991882270868, 99566408217116828, 109485351596724390, 105504044166670507, 99322214695067838, 107597858964689090, 107597870621708485, 107597875833524424, 105663527959374033, 105521239281281237, 105593278887024877, 109676337359108345, 109637393694097670, 109800832177252614, 105351724613755142, 107450819235010833, 105351835580819729, 1040164165

In [17]:
#Show all stages and regions for use in defining the dictionary of match importance for use in the elo algorithm
display(mh_df[['region', 'tournament', 'tournament_id', 'stage']].drop_duplicates(['region', 'stage']).sort_values('region'))


,region,tournament,tournament_id,stage
10745,All-Star Event,All-Star Event ase East 2020,105386292089445027,East
10751,All-Star Event,All-Star Event ase West 2020,105386292089445027,West
16950,Arabian League,Arabian League Spring Regular Season 2023,109545777182748074,Regular Season
17000,Arabian League,Arabian League Spring Playoffs 2023,109545777182748074,Playoffs
1949,CBLOL,CBLOL Split Regular Season 20201,103478354329449186,Regular Season
2020,CBLOL,CBLOL Split Promotion 20201,103478354329449186,Promotion
2025,CBLOL,CBLOL Split Playoffs 20201,103478354329449186,Playoffs
18913,CBLOL Academy,CBLOL Academy Academy Split Regular Season 1...,105562692792011783,Regular Season
18966,CBLOL Academy,CBLOL Academy Academy Split Playoffs 12021,105562692792011783,Playoffs
1918,College Championship,College Championship College Championship Play...,107898708099217418,Playoffs


In [18]:
def elo_change(blue_elo, red_elo, winner, region, stage, year):
    stage_values = {'Regionals': {
                        'Promotion Series': 15,
                        'Groups': 20,
                        'Regular Season': 20,
                        'Knockouts': 30,
                        'Playoffs': 30,
                        'Regional Qualifier': 30,
                        'Regional Finals': 35,
                    },
                    'MSI': {
                        'Play In Knockouts': 35,
                        'Round 1': 35,
                        'Round 2': 40,
                        'Knockouts': 45,
                        'Bracket Stage': 45,
                    },
                    'Worlds': {
                        'Play In Groups': 45,
                        'Play In Knockouts': 50,
                        'Groups': 55,
                        'Knockouts': 65,
                        'Playoffs': 65,
                    }
    }
    win_mul = 0.5**(2023 - int(year))
    loss_mul = 0.5**(2023 - int(year))
    if 'MSI' in region:
        match_type = 'MSI'
        if stage in ['Knockouts', 'Bracket Stage']:
            loss_mul *= 0.25
        else:
            loss_mul *= 0.5
    elif 'Worlds' in region:
        match_type = 'Worlds'
        if stage in ['Knockouts', 'Playoffs']:
            loss_mul *= 0
        else:
            loss_mul *= 0.25
    else:
        match_type = 'Regionals'
        if stage in ['Knockouts', 'Playoffs', 'Regional Finals', 'Regional Qualifier']:
            loss_mul *= 0.5
        else:
            loss_mul *= 1
    try:
        base_value = stage_values[match_type][stage]
    except: 
        base_value = 20
        print(region, stage)

    #1 for win, 0 for loss.
    blue_exp_res = 1 / (10**((-(blue_elo - red_elo))/600) + 1)
    red_exp_res = 1 / (10**((-(red_elo - blue_elo))/600) + 1)


    if winner == 100:
        blue_delta = base_value * (1-blue_exp_res) * win_mul
        red_delta = base_value * (0-red_exp_res) * loss_mul
    else:
        blue_delta = base_value * (0-blue_exp_res) * loss_mul
        red_delta = base_value * (1-red_exp_res) * win_mul

    print(f'Blue: {blue_elo:02} Red: {red_elo:02} Win: {winner:02} | Blue Delta: {blue_delta:02} Blue Pred: {blue_exp_res:02} | Red Delta: {red_delta:02} Red Pred: {red_exp_res:02} ')
    return blue_delta, red_delta


In [19]:
for match in mh_df.itertuples():
    blue_delta, red_delta = elo_change(elo_df.at[match.blue_team_id, 'elo'], 
                           elo_df.at[match.red_team_id, 'elo'],
                           match.winning_side,
                           match.region,
                           match.stage,
                           match.event_time[:4])
    elo_df.at[match.blue_team_id, 'elo'] += blue_delta
    elo_df.at[match.red_team_id, 'elo'] += red_delta

Blue: 1308.7149304690786 Red: 1302.7705296623851 Win: 100 | Blue Delta: 1.2357428175793737 Blue Pred: 0.5057028729682506 | Red Delta: -1.2357428175793737 Red Pred: 0.49429712703174944 
Blue: 1281.2470403632158 Red: 1299.8240881146378 Win: 100 | Blue Delta: 1.2945386720108614 Blue Pred: 0.48218453119565535 | Red Delta: -1.2945386720108614 Red Pred: 0.5178154688043446 
Blue: 1318.208437126162 Red: 1286.732799102395 Win: 100 | Blue Delta: 1.1745965200351685 Blue Pred: 0.5301613919859326 | Red Delta: -1.1745965200351687 Red Pred: 0.46983860801406746 
Blue: 1288.5665563744644 Red: 1000.0 Win: 100 | Blue Delta: 0.6208827765413225 Blue Pred: 0.751646889383471 | Red Delta: -0.6208827765413223 Red Pred: 0.24835311061652893 
Blue: 1309.7515512564084 Red: 1284.4905986353428 Win: 200 | Blue Delta: -1.310541649145941 Blue Pred: 0.5242166596583764 | Red Delta: 1.310541649145941 Red Pred: 0.4757833403416236 
Blue: 1298.5295494426268 Red: 1301.5347868448057 Win: 200 | Blue Delta: -1.2427919394339333 B

In [20]:
#Filter to only teams that have played games
final_elo = elo_df.loc[(teams)].sort_values('elo', ascending = False).reset_index()

#Filter to user specified(if any) tournament teams
tourn_list = [tourn.strip() for tourn in tournament_input.lower().split(',')] if len(tournament_input) > 0 else []
tourn_team_ids = []
for tourn in tourn_list:
    tourn_blue_ids = sorted(set(mh_df.loc[mh_df['tournament_id'].astype(str) == tourn, 'blue_team_id'].values))
    tourn_red_ids = sorted(set(mh_df.loc[mh_df['tournament_id'].astype(str) == tourn, 'red_team_id'].values))
    tourn_team_ids = sorted(set(tourn_team_ids + tourn_blue_ids + tourn_red_ids))
print(tourn_team_ids)
final_elo = final_elo.loc[final_elo['team_id'].isin(tourn_team_ids)] if len(tourn_team_ids) > 0 else final_elo
display(final_elo)

#Filter to user specified(if any) teams
team_list = [team.strip() for team in team_input.lower().split(',')] if len(team_input) > 0 else []
print(team_list)
final_elo = final_elo.loc[final_elo['team_id'].astype(str).isin(team_list)] if len(team_list) > 0 else final_elo
final_elo.index = final_elo.index.rename('Ranking')
final_elo.index = final_elo.index + 1
final_elo.to_csv('results.csv')
display(final_elo)

[]


,team_id,name,acronym,slug,region,elo
0,100205573495116443,Gen.G,GEN,geng,LCK,2123.053207
1,99566404579461230,kt Rolster,KT,kt-rolster,LCK,1994.402987
2,99566404852189289,Beijing JDG Intel Esports Club,JDG,jd-gaming,LPL,1967.737988
3,98767991853197861,T1,T1,t1,LCK,1957.797540
4,99566404853854212,Bilibili Gaming Pingan Bank,BLG,bilibili-gaming,LPL,1901.543658
5,99566404850008779,Suzhou LNG Esports,LNG,lng-esports,LPL,1815.579004
6,100725845018863243,Dplus Kia,DK,dwg-kia,LCK,1779.830973
7,100205573496804586,Hanwha Life Esports,HLE,hanwha-life-esports,LCK,1778.643072
8,98767991926151025,G2 Esports,G2,g2-esports,LEC,1738.845767
9,99566404853058754,WeiboGaming FAW AUDI,WBG,weibo-gaming,LPL,1734.978653


[]


,team_id,name,acronym,slug,region,elo
Ranking,,,,,,
1,100205573495116443,Gen.G,GEN,geng,LCK,2123.053207
2,99566404579461230,kt Rolster,KT,kt-rolster,LCK,1994.402987
3,99566404852189289,Beijing JDG Intel Esports Club,JDG,jd-gaming,LPL,1967.737988
4,98767991853197861,T1,T1,t1,LCK,1957.797540
5,99566404853854212,Bilibili Gaming Pingan Bank,BLG,bilibili-gaming,LPL,1901.543658
6,99566404850008779,Suzhou LNG Esports,LNG,lng-esports,LPL,1815.579004
7,100725845018863243,Dplus Kia,DK,dwg-kia,LCK,1779.830973
8,100205573496804586,Hanwha Life Esports,HLE,hanwha-life-esports,LCK,1778.643072
9,98767991926151025,G2 Esports,G2,g2-esports,LEC,1738.845767
